Group Name: AG 02.

Student Name (Student ID):

1. LI JIARU (A0332008U)

2. SHI YANCHUN (A0328710J)

3. XIAO XIAO (A0332142W)

4. JIN YINAN (A0327317E)

Group contributions: LI JIARU and SHI YANCHUN drafted all the answers in this Jupyter notebook. XIAO XIAO and JIN YINAN made all the improvement, and after discussion, we polished it together.

AI usage declaration: Due to our limited command of Python language, we partially used ChatGPT to write the search functions in Q1.b.1 based on the pseudocode and skeleton code given in class and to improve the grammar and expression in Q1.b.2. All the ideas and all other parts in this question are original. We are responsible for the content and quality of the submitted work.

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.


![Maze](Maze.jpg)


The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the form of a table.
 
3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.
 

Note 1: You must follow the problem formulation discussed in the class. A abstract class for Problem amd Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.). 

Note 2: The code should be written in a single jupyter notebook file.

In [1]:
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [2]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 


In [3]:
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
  
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [4]:
from collections import deque
import heapq
import itertools

FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

In [5]:
MAZE_ROWS = 16
MAZE_COLUMNS = 24
ACTIONS = {'UP', 'LEFT', 'RIGHT', 'DOWN'}

class Maze(Problem):

    def __init__(self, initial = None, goal = None, blocked = None, heuristic = 0): # Add blocked and heuristic to suit the problem
        Problem.__init__(self, initial = initial, goal = goal)
        self.blocked = blocked
        self.heuristic = heuristic

    def h(self, node):
        (x, y) = node
        (gx, gy) = self.goal
        dx = abs(gx - x)
        dy = abs(gy - y)
        if self.heuristic == 1:  # Weighted Manhattan distance
            return dx + 10 * dy
        elif self.heuristic == 2:  # Number of steps
            return dx + dy
        else:  # 0 by default, also for uniform-cost search
            return 0

    def action_cost(self, s, a, s1):
        if a in ('LEFT', 'RIGHT'):
            return 10
        elif a in ('UP', 'DOWN'):
            return 1

    def result(self, state, action):
        x, y = state
        if action == 'UP':
            return (x + 1, y)
        elif action == 'DOWN':
            return (x - 1, y)
        elif action == 'RIGHT':
            return (x, y + 1)
        elif action == 'LEFT':
            return (x, y - 1)

    def actions(self, state):
        x, y = state
        action = set()
        if x < MAZE_ROWS - 1 and (x + 1, y) not in self.blocked:
            action.add('UP')
        if x > 0 and (x - 1, y) not in self.blocked:
            action.add('DOWN')
        if y < MAZE_COLUMNS - 1 and (x, y + 1) not in self.blocked:
            action.add('RIGHT')
        if y > 0 and (x, y - 1) not in self.blocked:
            action.add('LEFT')
        return action

### Testing the model

In [6]:
#Testing Maze class
#Note that we perform additional tests to validate the modeling

#An instance of Maze problem; initialize the Maze class with appropriate values
blocked = {(6, 9), (7, 9), (8, 9), (9, 9), (10, 9), (14, 9),
           (10, 10), (11, 10), (12, 10), (13, 10),
           (10, 12), (11, 12), (10, 13), (11, 13)}
m1 = Maze(initial = (8, 10), goal = (11, 9), blocked = blocked)

# Testing the methods in Maze class
assert m1.result((4,12),'UP') == (5,12)
assert m1.action_cost((4,12),'UP',(5,12)) == 1
assert m1.actions((4,12)) == {'UP', 'LEFT', 'RIGHT', 'DOWN'}

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

In [7]:
#Your code for breadth-first search

def bfs(problem):
    node = Node(problem.initial)
    if problem.is_goal(node.state): 
        return node, 0, 1, node.path_cost
    frontier = deque([node])
    explored = set([node.state])
    expanded, max_frontier = 0, 1
    while frontier:
        max_frontier = max(max_frontier, len(frontier))
        node = frontier.popleft()
        expanded += 1
        for child in expand(problem, node):
            if child.state in explored: 
                continue
            if problem.is_goal(child.state):
                return child, expanded, max_frontier, child.path_cost
            explored.add(child.state)
            frontier.append(child)
    return failure, expanded, max_frontier, math.inf

In [8]:
#Your code for depth-first search with cycle-check

def dfs(problem, limit = None):
    if limit is None:
        limit = MAZE_ROWS * MAZE_COLUMNS
    start = Node(problem.initial)
    expanded, max_frontier = 0, 0
    explored = set()
    def rec(n, depth, on_path):
        nonlocal expanded, max_frontier, explored
        expanded += 1
        max_frontier = max(max_frontier, len(on_path))
        if problem.is_goal(n.state):
            return n
        if depth == limit:
            return None
        children = list(expand(problem, n))
        children.sort(key=lambda c: (c.action in ('LEFT', 'RIGHT'), c.action))
        for child in children:
            if child.state in on_path or child.state in explored:
                continue
            on_path.add(child.state)
            res = rec(child, depth + 1, on_path)
            on_path.remove(child.state)
            if res is not None:
                return res
        explored.add(n.state)
        return None
    result = rec(start, 0, set([start.state]))
    return (result or failure), expanded, max_frontier, (result.path_cost if result else math.inf)

In [9]:
#Your code for iterative Deepening search with cycle-check

def ids(problem, max_depth = None):
    if max_depth is None:
        max_depth = MAZE_ROWS * MAZE_COLUMNS
    total_expanded, max_frontier_seen = 0, 0
    for limit in range(max_depth + 1):
        res, exp, mf, cost = dfs(problem, limit = limit)
        total_expanded += exp
        max_frontier_seen = max(max_frontier_seen, mf)
        if res is not failure:
            return res, total_expanded, max_frontier_seen, cost
    return failure, total_expanded, max_frontier_seen, math.inf

In [10]:
#Your code for uniform-cost search

def ucs(problem):
    _counter = itertools.count()
    start = Node(problem.initial)
    frontier = [(start.path_cost, next(_counter), start)]
    best_g = {start.state: 0}
    expanded, max_frontier = 0, 1
    while frontier:
        max_frontier = max(max_frontier, len(frontier))
        _, _, node = heapq.heappop(frontier)
        expanded += 1
        if problem.is_goal(node.state): 
            return node, expanded, max_frontier, node.path_cost
        for child in expand(problem, node):
            g = child.path_cost
            if g < best_g.get(child.state, math.inf):
                best_g[child.state] = g
                heapq.heappush(frontier, (g, next(_counter), child))
    return failure, expanded, max_frontier, math.inf

In [11]:
#Your code for A* Search

def astar(problem):
    _counter = itertools.count()
    start = Node(problem.initial)
    frontier = [(problem.h(start.state), next(_counter), start)]
    best_g = {start.state: 0}
    expanded, max_frontier = 0, 1
    while frontier:
        max_frontier = max(max_frontier, len(frontier))
        _, _, node = heapq.heappop(frontier)
        expanded += 1
        if problem.is_goal(node.state): 
            return node, expanded, max_frontier, node.path_cost
        children = list(expand(problem, node))
        children.sort(key = lambda c: (c.action in ('LEFT', 'RIGHT'), c.action))
        for child in children:
            g = child.path_cost
            if g < best_g.get(child.state, math.inf):
                best_g[child.state] = g
                f = g + problem.h(child.state)
                heapq.heappush(frontier, (f, next(_counter), child))
    return failure, expanded, max_frontier, math.inf

In [12]:
#Your code for greedy-best first search

def gbfs(problem):
    _counter = itertools.count()
    start = Node(problem.initial)
    frontier = [(problem.h(start.state), next(_counter), start)]
    visited = set()
    expanded, max_frontier = 0, 1
    while frontier:
        max_frontier = max(max_frontier, len(frontier))
        _, _, node = heapq.heappop(frontier)
        if problem.is_goal(node.state): 
            return node, expanded, max_frontier, node.path_cost
        if node.state in visited:
            continue
        visited.add(node.state)
        expanded += 1
        children = list(expand(problem, node))
        children.sort(key = lambda c: (c.action in ('LEFT', 'RIGHT'), c.action))
        for child in children:
            if child.state not in visited:
                heapq.heappush(frontier, (problem.h(child.state), next(_counter), child))
    return failure, expanded, max_frontier, math.inf

2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the following table.

In [13]:
def results(problem):
    result = []
    result.append(("BFS", *bfs(problem)))
    result.append(("DFS", *dfs(problem)))
    result.append(("IDS", *ids(problem)))
    result.append(("UCS", *ucs(problem)))
    result.append(("A*", *astar(problem)))
    result.append(("GBFS", *gbfs(problem)))
    return result

for name, sol, expanded, frontier, cost in results(m1):
    print(f"{name:4s} expanded={expanded:4d} frontier={frontier:3d} cost={cost:2d}")

BFS  expanded= 159 frontier= 30 cost=39
DFS  expanded= 160 frontier=149 cost=87
IDS  expanded=1097 frontier= 15 cost=41
UCS  expanded=  91 frontier= 34 cost=39
A*   expanded=  92 frontier= 34 cost=39
GBFS expanded= 188 frontier= 61 cost=39



 Algorithm             |Number of expanded nodes  | Maximum Frontier Size  |  Path Cost                  |
|:---------------------|:-------------------------|:-----------------------|:----------
| Breadth-First Search | 159                      | 30                     | 39
| DFS with cycle check | 160                      | 149                    | 87
| IDS with cycle check | 1097                     | 15                     | 41
| UCS                  | 91                       | 34                     | 39
| A* Search            | 92                       | 34                     | 39
| GBFS                 | 188                      | 61                     | 39

3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.



In [14]:
m1_1 = Maze(initial = (8, 10), goal = (11, 9), blocked = blocked, heuristic = 1) # Same problem with heuristic: weighted Manhattan distance
m1_2 = Maze(initial = (8, 10), goal = (11, 9), blocked = blocked, heuristic = 2) # Same problem with heuristic: number of steps

sol, expanded, frontier, cost = astar(m1_1)
print(f"A* with weighted Manhattan distance: expanded={expanded} frontier={frontier} cost={cost}")
sol, expanded, frontier, cost = astar(m1_2)
print(f"A* with number of steps: expanded={expanded} frontier={frontier} cost={cost}")
sol, expanded, frontier, cost = gbfs(m1_1)
print(f"GBFS with weighted Manhattan distance: expanded={expanded} frontier={frontier} cost={cost}")
sol, expanded, frontier, cost = gbfs(m1_2)
print(f"GBFS with number of steps: expanded={expanded} frontier={frontier} cost={cost}")

A* with weighted Manhattan distance: expanded=35 frontier=29 cost=39
A* with number of steps: expanded=66 frontier=34 cost=39
GBFS with weighted Manhattan distance: expanded=18 frontier=26 cost=39
GBFS with number of steps: expanded=22 frontier=22 cost=61


From the table above we can see that the weighted Manhattan distance heuristic significantly outperforms the step count heuristic in both informed search algorithms:

A* Search:
- A* (Weighted Manhattan): 35 expanded nodes, 29 max frontier, cost 39
- A* (Step count heuristic): 66 expanded nodes, 34 max frontier, cost 39  

GBFS Search:
- GBFS (Weighted Manhattan): 18 expanded nodes, 26 max frontier, cost 39
- GBFS (Step count heuristic): 22 expanded nodes, 22 max frontier, cost 61

We can then conclude that:

1. A* Performance: Weighted Manhattan distance reduces expanded nodes by 47% compared to step count heuristic (35 vs 66), while maintaining optimal solution quality
2. GBFS Performance: Weighted Manhattan distance is not only more efficient (18 vs 22 expanded nodes) but also guarantees optimal solution (cost 39), while step count heuristic finds suboptimal solution (cost 61)
3. Heuristic Quality: The step count heuristic systematically underestimates horizontal movement costs (10x difference), misleading both A* and GBFS algorithms

The weighted Manhattan distance heuristic is the best performing heuristic because it perfectly matches the problem's cost model (vertical = 1, horizontal = 10), providing superior efficiency in A* and both efficiency and solution quality in GBFS.